## Text Generation Inference using Online Endpoints

This sample shows how to deploy `text-generation` type models to an online endpoint for inference.

### Task
`text-generation`  is the task of producing new text. These models can, for example, fill in incomplete text or paraphrase. Some common applications of text generation are code generation and story generation.

### Model
Models that can perform the `text-generation` task are tagged with `task: text-generation`. We will use the `gpt2` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import_model_into_registry.ipynb) and then use them for inference. 

### Inference data
We will use the [book corpus](https://huggingface.co/datasets/bookcorpus) dataset.

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Download and prepare data for inference. 
* Deploy the model for real time inference.
* Test the endpoint
* Clean up resources.

### 1. Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [1]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
    credential,
    subscription_id= "<SUBSCRIPTION_ID>",
    resource_group_name= "<RESOURCE_GROUP>",
    workspace_name= "<WORKSPACE_NAME>",
)
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml"
# registry_ml_client = MLClient(credential, registry_name="azureml-msr")

In [ ]:
workspace_ml_client

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f6336f98040>,
         subscription_id=80c77c76-74ba-4c8c-8229-4c3b2957990c,
         resource_group_name=huggingface-registry-test1,
         workspace_name=test-eastus)

### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `text-generation` task. In this example, we use the `gpt2` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [2]:
model_name = "gpt2"
version_list = list(registry_ml_client.models.list(model_name))
if len(version_list) == 0:
    print("Model not found in registry")
else:
    model_version = version_list[0].version
    foundation_model = registry_ml_client.models.get(model_name, model_version)
    print(
        "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
            foundation_model.name, foundation_model.version, foundation_model.id
        )
    )



Using model name: microsoft-phi-2, version: 2, id: azureml://registries/azureml-msr/models/microsoft-phi-2/versions/2 for inferencing


### 3. Download and prepare data for inference.

The next few cells show basic data preparation:
* Visualize some data rows
* Save few samples in the format that can be passed as input to the online-inference endpoint.

In [6]:
# Download a small sample of the dataset into the ./book-corpus-dataset directory
%run ./book-corpus-dataset/download-dataset.py --download_dir ./book-corpus-dataset

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading train split of bookcorpus dataset...


In [7]:
# load the ./book-corpus-dataset/train.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd

pd.set_option(
    "display.max_colwidth", 0
)  # set the max column width to 0 to display the full text
train_df = pd.read_json("./book-corpus-dataset/train.jsonl", lines=True)
train_df.head(2)

,text
0,"thrusting out her hand , megan smiled warmly at him ."
1,"`` you 're certainly a very attractive interrogator , '' he said ."


### 4. Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [8]:
import time, sys
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    ProbeSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "text-generation-oss-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", for text-generation task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [9]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS3_v2",
    instance_count=2,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=1000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=1000,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

Check: endpoint text-generation-oss-1702044733 exists


.........................................................................................................................................................................................................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://text-generation-oss-1702044733.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://text-generation-oss-1702044733.eastus.inference.ml.azure.com/swagger.json', 'name': 'text-generation-oss-1702044733', 'description': 'Online endpoint for gpt2, for text-generation task', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/80c77c76-74ba-4c8c-8229-4c3b2957990c/resourcegroups/huggingface-registry-test1/providers/microsoft.machinelearningservices/workspaces/test-eastus/onlineendpoints/text-generation-oss-1702044733', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/80c77c76-74ba-4c8c-8229-4c3b2957990c/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:52e6e75c-0813-4c25-90fc-7e8ba6e37dc9:bf2e1ec3-f5b6-4d6c-8c1c-a2350579ba9d?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscr

### 5. Test the endpoint with sample data

We will fetch some sample data from the test dataset and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [10]:
import json
import os

# read the ./book-corpus-dataset/train.jsonl file into a pandas dataframe
df = pd.read_json("./book-corpus-dataset/train.jsonl", lines=True)
# escape single and double quotes in the text column
df["text"] = df["text"].str.replace("'", "\\'").str.replace('"', '\\"')
# pick 1 random row
sample_df = df.sample(1)
# pick the params from the input sample on the model card
params = {"top_p": 0.9, "temperature": 0.2, "max_new_tokens": 50, "do_sample": True, "return_full_text": True}
# save the json object to a file named sample_score.json in the ./book-corpus-dataset folder
test_json = {"input_data": sample_df["text"].tolist(), "params": params}
# save the json object to a file named sample_score.json in the ./book-corpus-dataset folder
with open(os.path.join(".", "book-corpus-dataset", "sample_score.json"), "w") as f:
    json.dump(test_json, f)
sample_df.head()

,text
0,"thrusting out her hand , megan smiled warmly at him ."


In [11]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="./book-corpus-dataset/sample_score.json",
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe
response_df = pd.read_json(response)
response_df.head()

raw response: 
 ["thrusting out her hand , megan smiled warmly at him . \"You're not like an evil witch, but that's what you did to me. It's why they call you, after all. And when you ask for anything,"] 



,0
0,"thrusting out her hand , megan smiled warmly at him . ""You're not like an evil witch, but that's what you did to me. It's why they call you, after all. And when you ask for anything,"


### 6. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()